### Step 1: Import the Packages

In [1]:
import pandas as pd
import numpy as np
import os
from google.cloud import bigquery
from google.cloud import bigquery_storage
from scipy.stats import wilcoxon
import warnings
warnings.filterwarnings(action="ignore")

g:\My Drive\APAC\Autopricing\Switchback Testing\switchback_test_dag\venv_sb\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.11) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


### Step 2.1: Instantiate a BQ client

In [2]:
client = bigquery.Client(project="logistics-data-staging-flat")
bqstorage_client = bigquery_storage.BigQueryReadClient()

### Step 2.2: Import the SQL queries and run them

In [3]:
sql_queries = [i for i in os.listdir(path=os.getcwd() + "/sql_queries") if i.endswith(".sql")]

for i in sql_queries:
    with open(file=os.getcwd() + f"/sql_queries/{i}", mode="r") as sql:
        client.query(query=sql.read()).result()
        sql.close()

### Step 2.3: Pull the order data from the final dataset

In [3]:
df = client\
    .query(query="""SELECT * FROM `dh-logistics-product-ops.pricing.ab_test_individual_orders_cleaned_lb_rollout_tests`""")\
    .result()\
    .to_dataframe(bqstorage_client=bqstorage_client, progress_bar_type="tqdm")

Downloading: 100%|██████████| 4043531/4043531 [1:07:18<00:00, 1001.23rows/s] 


### Step 2.4: Create a dataset of the business KPIs analyzed on the treatment scope level

In [244]:
# Make a copy of df
df_copy = df.copy()

In [247]:
# Change the data type of some columns to an appropriate type
df_copy[["test_id", "dps_travel_time_fee_local", "gfv_local", "gmv_local", "commission_local"]] = df_copy[["test_id", "dps_travel_time_fee_local", "gfv_local", "gmv_local", "commission_local"]]\
    .apply(pd.to_numeric, errors="ignore")

# Find the first and last date of an experiment and eliminate those two dates
df_dates = df_copy.groupby(["region", "entity_id", "test_id", "test_name"], as_index=False)["created_date_utc"].agg({"min_date": "min", "max_date": "max"})

# Join df_dates to df
df_copy = pd.merge(left=df_copy, right=df_dates[["entity_id", "test_id", "min_date", "max_date"]], how="left", on=["entity_id", "test_id"])

# Filter out the orders that occurred on the first or last day of the experiment
df_copy = df_copy[(df_copy["created_date_utc"] > df_copy["min_date"]) & (df_copy["created_date_utc"] < df_copy["max_date"])]

# Create a KPI list
kpi_list = ["actual_df_paid_by_customer", "dps_travel_time_fee_local", "gfv_local", "gmv_local", "commission_local", "revenue_local", "delivery_costs_local", "gross_profit_local"]

In [261]:
# Create a function that aggregates the metrics and computes the p-values on any granularity (experiment, treatment/non-treatment scope, and target group)
def pval_func(granularity): # granularity can be "experiment", "treatment", "non_treatment", "target_group"
    if granularity == "experiment":
        grouping_vars_for_agg = ["region", "entity_id", "test_id", "test_name", "variant", "created_date_utc"]
    elif granularity == "treatment" or granularity == "non_treatment":
        grouping_vars_for_agg = ["region", "entity_id", "test_id", "test_name", "is_in_treatment", "variant", "created_date_utc"]
    elif granularity == "target_group":
        grouping_vars_for_agg = ["region", "entity_id", "test_id", "test_name", "target_group_bi", "variant", "created_date_utc"]

    # Calculate the total KPIs per entity_id, test_id, variant, and date
    df_copy_agg = df_copy.groupby(grouping_vars_for_agg)[kpi_list].sum()

    # Add the order count to df_copy_agg
    df_copy_agg["order_count"] = df_copy.groupby(grouping_vars_for_agg)["order_id"].nunique()

    # Reset the index of df_copy_agg
    df_copy_agg.reset_index(inplace=True)

    return df_copy_agg

# Create multiple data frames containing the right data cut for each granularity
df_exp = pval_func(granularity="experiment")
df_treatment = pval_func(granularity="treatment")
df_treatment = df_treatment[df_treatment["is_in_treatment"]==True]
df_non_treatment = pval_func(granularity="non_treatment")
df_non_treatment = df_non_treatment[df_non_treatment["is_in_treatment"]==False]
df_target_group = pval_func(granularity="target_group")

# Create a data frame containing the distinct combinations of test_name and variant. This is the data frame that will receive the p-values from the wilcoxon function
test_var_list = ["test_name", "variant"]
df_pval = df_exp[df_exp["variant"] != "Control"][test_var_list].drop_duplicates().reset_index(drop=True) # We could use any "df" here, not only df_exp
df_pval["key"] = 1

# Create a data frame containing all granularities
granularity_list = ["experiment", "treatment", "non_treatment"]
tg_list = df_copy["target_group_bi"].dropna().unique().tolist()
granularity_list.extend(tg_list)
df_granularity = pd.DataFrame(granularity_list, columns=["granularity"])
df_granularity["key"] = 1

# Create a data frame containing the names of the KPIs
kpi_list_with_orders = ["order_count"]
kpi_list_with_orders.extend(kpi_list)
df_kpi_list_with_order_count = pd.DataFrame(kpi_list_with_orders, columns=["kpi"])
df_kpi_list_with_order_count["key"] = 1

# Perfrom a cross join to get the distinct combinations of test_name, variant, and KPI
df_pval = pd.merge(left=df_pval, right=df_granularity, how="left", on="key")
df_pval = pd.merge(left=df_pval, right=df_kpi_list_with_order_count, how="left", on="key").drop("key", 1)

# Add the p-value column. The None values will be replaced with actual values by the Wilcoxon function
df_pval["pval"] = None

# Iterate over each test_name, variant, and KPI and calculate the p-value between variation(x) and control
for i in range(len(df_pval)):
    test_iter = df_pval.loc[i, "test_name"]
    variant_iter = df_pval.loc[i, "variant"]
    granularity_iter = df_pval.loc[i, "granularity"]
    kpi_iter = df_pval.loc[i, "kpi"]

    # Choosing the right data cuts based on the granularity
    if granularity_iter == "experiment":
        df_calc = df_exp.copy()
    elif granularity_iter == "treatment":
        df_calc = df_treatment.copy()
    elif granularity_iter == "non_treatment":
        df_calc = df_non_treatment.copy()
    elif granularity_iter == "Target Group 1":
        df_calc = df_target_group[df_target_group["target_group_bi"] == "Target Group 1"].copy()
    elif granularity_iter == "Target Group 2":
        df_calc = df_target_group[df_target_group["target_group_bi"] == "Target Group 2"].copy()
    elif granularity_iter == "Target Group 3":
        df_calc = df_target_group[df_target_group["target_group_bi"] == "Target Group 3"].copy()
    elif granularity_iter == "Target Group 4":
        df_calc = df_target_group[df_target_group["target_group_bi"] == "Target Group 4"].copy()

    try:
        df_pval.loc[i, "pval"] = wilcoxon(
            x=df_calc[(df_calc["variant"] == variant_iter) & (df_calc["test_name"] == test_iter)][kpi_iter],
            y=df_calc[(df_calc["variant"] == "Control") & (df_calc["test_name"] == test_iter)][kpi_iter],
            zero_method="wilcox",
            correction=False,
            alternative="two-sided",
            method="auto",
            nan_policy="omit"
        ).pvalue
    except ValueError as err: # Sometimes, samples x and y are not equal in length due to missing data. If that's the case, set the p-value to None
        df_pval.loc[i, "pval"] = None

### Step 2.5: Upload df_pval to BigQuery

In [ ]:
# Set the job_config to overwrite the data in the table
job_config = bigquery.LoadJobConfig()
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

# Upload the p-values data frame to BQ
job = client.load_table_from_dataframe(
    dataframe=df_pval,
    destination="dh-logistics-product-ops.pricing.p_vals_lb_rollout_tests",
    job_config=job_config
)

# Wait for the load job to complete
job.result()

In [266]:
pd.set_option("display.max_rows", 100)